In [ ]:
import os, sys
import ROOT
from ROOT import fcllite
from ROOT import geo2d
from larcv import larcv
import cv2
pygeo = geo2d.PyDraw()
from ROOT import larocv
from ROOT import std
from ROOT import cv
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
matplotlib.rcParams['font.size']=20
matplotlib.rcParams['font.family']='serif'
%matplotlib inline
import numpy as np

In [ ]:
proc = larcv.ProcessDriver('ProcessDriver')
CFG="../cfg/prod_fullchain_ssnet_combined_test.cfg"
#CFG="../cfg/prod_fullchain_ssnet_combined.cfg"

preprocessed=False

print "Loading config... ",CFG
proc.configure(CFG)
flist=ROOT.std.vector('std::string')()
flist.push_back("/Users/vgenty/Desktop/intrinsic_nue/out_pyroi/circle_10_pi_10_unique_yes_xs_1/valid_shower_pt/b.root")
proc.override_input_file(flist)
proc.override_output_file("/tmp/cacca.root")
proc.override_ana_file("/tmp/test.root")

vinroi_id  = proc.process_id("VertexInROI")
reco_id    = proc.process_id("LArbysImage")
larbysimg  = proc.process_ptr(reco_id)

proc.initialize()

In [ ]:
event=0
proc.batch_process(event,1)

In [ ]:
mgr=larbysimg.Manager()  

In [ ]:
img_v  = [None,None,None]
oimg_v = [None,None,None]
track_img_v  = [None,None,None]
otrack_img_v = [None,None,None]
shower_img_v  = [None,None,None]
oshower_img_v = [None,None,None]

for ix in xrange(3):
    oimg_v[ix]        = pygeo.image(mgr.OriginalInputImages(0)[ix])
    otrack_img_v[ix]  = pygeo.image(mgr.OriginalInputImages(1)[ix])
    oshower_img_v[ix] = pygeo.image(mgr.OriginalInputImages(2)[ix])
    img_v[ix]         = pygeo.image(mgr.InputImages(0)[ix])
    track_img_v[ix]   = pygeo.image(mgr.InputImages(1)[ix])
    shower_img_v[ix]  = pygeo.image(mgr.InputImages(2)[ix])

### SSNet Output

In [ ]:
for plane in xrange(len(track_img_v)):
    oshower_img = np.where(oshower_img_v[plane]>10.0,85.0 ,0.0).astype(np.uint8)
    otrack_img  = np.where(otrack_img_v[plane] >10.0,160.0,0.0).astype(np.uint8)
    shower_img  = np.where(shower_img_v[plane] >10.0,85.0 ,0.0).astype(np.uint8)
    track_img   = np.where(track_img_v[plane]  >10.0,160.0,0.0).astype(np.uint8)
    fig,ax=plt.subplots(figsize=(10,10))
    oimg = oshower_img + otrack_img
    img  = shower_img  + track_img
    ax.set_xlabel('Time [6 ticks]',fontsize=20)
    ax.set_ylabel('Wire',fontsize=20)
    ax.imshow(img,cmap='jet',interpolation='none',vmin=0.,vmax=255.)
    ax.tick_params(labelsize=20)
    plt.tight_layout()
    plt.show()

### Track Vertex Seeds

In [ ]:
colors=['red','green','blue','orange','magenta','cyan','pink']
colors*=10
dm=mgr.DataManager()
algoid=dm.ID("trackvertexseeds")
print "Algorithm ID:",algoid
for plane in xrange(3):
    fig,ax=plt.subplots(figsize=(12,12),facecolor='w')
    shape_img1 = track_img_v[plane]
    shape_img2 = shower_img_v[plane]
    shape_img1=np.where(shape_img1>10.0,255.0,0.0).astype(np.uint8)
    shape_img2=np.where(shape_img2>10.0,100.0,0.0).astype(np.uint8)
    shape_img=shape_img1+shape_img2
    ax.imshow(shape_img,cmap='Greys',interpolation='none')
    nz_pixels=np.where(shape_img>0.0)
    ix=0

    ####################################
    vertex_seeds_v = dm.Data(1,plane).as_vector()
    cluscomp_v     = dm.Data(1,3+plane).as_vector()

    #plot the atomics
    for cluscomp_id in xrange(cluscomp_v.size()):
        cluscomp = cluscomp_v[cluscomp_id]
        for atomic_id in xrange(cluscomp.size()):
            atomic = cluscomp[atomic_id]
            pts=[[atomic[p_id].x,atomic[p_id].y] for p_id in xrange(atomic.size())]
            if len(pts)==0: continue
            pts.append(pts[0])
            pts=np.array(pts)
            plt.plot(pts[:,0],pts[:,1],'-',lw=3,color=colors[ix],alpha = 0.8)
            ix+=1

    pts_v = np.array([[vertex_seeds_v[i].x,
                       vertex_seeds_v[i].y] for i in xrange(vertex_seeds_v.size())])
    if pts_v.size>0:
        plt.plot(pts_v[:,0],pts_v[:,1],'*',markersize=20,color='yellow')

    ####################################
    plt.tight_layout()
    try:
        ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
    except ValueError:
        pass

    plt.xlabel('Time [6 ticks]',fontsize=20)
    plt.ylabel('Wire',fontsize=20)
    plt.tick_params(labelsize=20)
    ax.set_aspect(0.8)
    ax.grid()
    plt.show()

### Track Vertex Estimate

In [ ]:
colors=['red','green','blue','orange','magenta','cyan','pink']
colors*=10
dm=mgr.DataManager()
algoid=dm.ID("trackvertexestimate")
print "Algorithm ID:",algoid
for plane in xrange(3):
    fig,ax=plt.subplots(figsize=(12,12),facecolor='w')
    shape_img1 = track_img_v[plane]
    shape_img2 = shower_img_v[plane]
    shape_img1=np.where(shape_img1>10.0,255.0,0.0).astype(np.uint8)
    shape_img2=np.where(shape_img2>10.0,100.0,0.0).astype(np.uint8)
    shape_img=shape_img1+shape_img2
    ax.imshow(shape_img,cmap='Greys',interpolation='none')
    nz_pixels=np.where(shape_img>0.0)
    ix=0
    
    ####################################
    vertex_vv  = dm.Data(algoid,0).as_vector()
    cluscomp_v = dm.Data(algoid-1,3+plane).as_vector()
    for cluscomp_id in xrange(cluscomp_v.size()):
        cluscomp = cluscomp_v[cluscomp_id]
        for atomic_id in xrange(cluscomp.size()):
            atomic = cluscomp[atomic_id]
            pts=[[atomic[p_id].x,atomic[p_id].y] for p_id in xrange(atomic.size())]
            if len(pts)==0: continue
            pts.append(pts[0])
            pts=np.array(pts)
            plt.plot(pts[:,0],pts[:,1],'-o',lw=3,color=colors[ix], alpha = 0.5)
            ix+=1

    pts_v = np.array([[vertex_vv[i].cvtx2d_v[plane].center.x,
                       vertex_vv[i].cvtx2d_v[plane].center.y] for i in xrange(vertex_vv.size())])
    if pts_v.size>0:
        print "\t<=== Track Vertex Candidates @ ===>\n",pts_v
        ax.plot(pts_v[:,0],pts_v[:,1],'*',markersize=30,color='cyan')
    ####################################
    
    plt.tight_layout()
    try:
        ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
    except ValueError:
        pass

    plt.xlabel('Time [6 ticks]',fontsize=20)
    plt.ylabel('Wire',fontsize=20)
    plt.tick_params(labelsize=20)
    ax.set_aspect(0.8)
    ax.grid()
    plt.show()

### Angular Metric

In [ ]:
algoid=dm.ID("trackvertexestimate")
alg=mgr.GetClusterAlg(algoid).Algo()

tickscore0_y=[]
tickscore0_x=[]

score0_v = alg.TimeBinnedScore0Mean()
for idx in xrange(score0_v.size()):
    v = score0_v[idx]
    tickscore0_y.append(v)
    tickscore0_x.append(idx*1 + alg.TimeBinMin())

tickscore1_y=[]
tickscore1_x=[]
score1_v = alg.TimeBinnedScore1Mean()
for idx in xrange(score1_v.size()):
    v = score1_v[idx]
    tickscore1_y.append(v)
    tickscore1_x.append(idx*1 + alg.TimeBinMin())

tickscore0_x = np.array(tickscore0_x)
tickscore0_y = np.array(tickscore0_y)
tickscore1_x = np.array(tickscore1_x)
tickscore1_y = np.array(tickscore1_y)
ymin = tickscore0_y.min()
ymax = tickscore0_y.max()
if ymin > tickscore1_y.min(): ymin = tickscore1_y.min()
if ymax < tickscore1_y.max(): ymax = tickscore1_y.max()

fig,ax = plt.subplots(figsize=(16,8),facecolor='w')
ax.plot(tickscore0_x,tickscore0_y,marker='o',linestyle='-',color='red',markersize=10)
ax.plot(tickscore1_x,tickscore1_y,marker='o',linestyle='--',
         markeredgewidth=1,markeredgecolor='blue',markerfacecolor='None',markersize=10)


minimum_v  = alg.TimeBinnedScoreMinIndex()
minrange_v = alg.TimeBinnedScoreMinRange()
for idx in xrange(minimum_v.size()):
    xval = tickscore0_x[minimum_v[idx]]
    ax.plot([xval,xval],[0,360],marker='',linestyle='--',color='black',linewidth=2)
    xstart = tickscore0_x[minrange_v[idx].first]
    xend   = tickscore0_x[minrange_v[idx].second]
    ax.axvspan(xstart,xend, alpha=0.3, color='orange')

ax.set_xlabel('Time [6 ticks]',fontsize=20,fontweight='bold')
ax.set_ylabel('Summed Angle Difference [deg.]',fontsize=20,fontweight='bold')
ax.tick_params(labelsize=20)
plt.tight_layout()
ax.set_ylim(-1,ymax*1.1)
ax.set_xlim(tickscore0_x.min(),tickscore0_x.max())
ax.grid()
plt.show()


### Track Type Particle Clustering

In [ ]:
assman=dm.AssManager()
colors=['red','green','blue','orange','magenta','cyan','pink']
colors*=10
dm=mgr.DataManager()
algoid=dm.ID("trk_vertexparticleclustermaker")
vtx_data=dm.Data(algoid-2,0).as_vector()

####################################
for ix,vtx in enumerate(vtx_data):
    
    print "<==========================================================================>"
    print "<===================Start Vertex3D number ",ix," ==========================>"
    cvtx_v = vtx.cvtx2d_v
    
    for plane in xrange(3):
        cvtx = cvtx_v[plane]
        fig,ax = plt.subplots(figsize=(12,12),facecolor='w')
        shape_img1 = track_img_v[plane]
        shape_img2 = shower_img_v[plane]
        shape_img1=np.where(shape_img1>10.0,255.0,0.0).astype(np.uint8)
        shape_img2=np.where(shape_img2>10.0,100.0,0.0).astype(np.uint8)
        shape_img=shape_img1+shape_img2
        ax.imshow(shape_img,cmap='Greys',interpolation='none')
        nz_pixels=np.where(shape_img>0.0)

        par_data=dm.Data(algoid,plane)

        ass_t = np.array(assman.GetManyAss(vtx,par_data.ID()))
        if ass_t.size==0:continue

        par_data_v=par_data.as_vector()
        for id_ in ass_t:
            ctor=[[pt.x,pt.y] for pt in par_data_v[id_]._ctor]
            if len(ctor)==0:continue
            ctor.append(ctor[0])
            ctor=np.array(ctor)
            ax.plot(ctor[:,0],ctor[:,1],'-',lw=4)

        ax.plot(cvtx.center.x,
                cvtx.center.y,
                'o',color='red',markersize=10)
        
        circle_=patches.Circle((cvtx.center.x,cvtx.center.y),
                                cvtx.radius,fc='none',ec='cyan',lw=5,alpha=0.5)
        
        print "Vertex",ix,"plane @",plane,"..."
        for xs in cvtx.xs_v:
            print  "xs @ [",xs.pt.x,",",xs.pt.y,"]"
            ax.plot(xs.pt.x,xs.pt.y,'o',color='orange',markersize=10,alpha=0.7)
            
        ax.add_patch(circle_)
        ax.set_aspect(1.0)
        plt.tight_layout()
        ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
        ax.set_xlabel('Time [6 ticks]',fontsize=20)
        ax.set_ylabel('Wire',fontsize=20)
        ax.tick_params(labelsize=20)
        plt.show()
    print "<===================End   Vertex3D number ",ix," ==========================>"
    print "<==========================================================================>"
####################################

### VertexScanner3D

In [ ]:
# dm=mgr.DataManager()
# colors=['red','green','blue','orange','magenta','cyan','pink']
# colors*=10

# dm=mgr.DataManager()
# algoid=dm.ID("vertexscanner3d")
# data=dm.Data(algoid,0)
# vtxseed3d_v=data.as_vector()
# print vtxseed3d_v.size()

# for vtxid,vtx in enumerate(vtxseed3d_v):
#     print "<=============================================================================>"
#     print "<===================Start combined vertex ",vtxid," ==========================>"
#     for plane in xrange(3):
#         fig,ax = plt.subplots(figsize=(12,12),facecolor='w')
#         shape_img1 = track_img_v[plane]
#         shape_img2 = shower_img_v[plane]
#         shape_img1=np.where(shape_img1>10.0,255.0,0.0).astype(np.uint8)
#         shape_img2=np.where(shape_img2>10.0,100.0,0.0).astype(np.uint8)
#         shape_img=shape_img1+shape_img2
#         ax.imshow(shape_img,cmap='Greys',interpolation='none')
#         nz_pixels=np.where(shape_img>0.0)
#         print vtx.vtx2d_v.size()
#         ax.plot(vtx.vtx2d_v[plane].x,vtx.vtx2d_v[plane].y,'*',color='cyan',markersize=20)
        
#         ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
#         ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
#         ax.set_title("Vertex Type: %d"%vtx.type)
#         print "<=============================================================================>"
#         print "<===================End combined vertex ",vtxid," ==========================>"
#         plt.show()



### ShowerVertexScan

In [ ]:
dm=mgr.DataManager()
colors=['red','green','blue','orange','magenta','cyan','pink']
colors*=10

dm=mgr.DataManager()
algoid=dm.ID("showervertexscan")
data=dm.Data(algoid,0)
vtxseed3d_v=data.as_vector()
print vtxseed3d_v.size()

for vtxid,vtx in enumerate(vtxseed3d_v):
    print "<=============================================================================>"
    print "<===================Start combined vertex ",vtxid," ==========================>"
    for plane in xrange(3):
        fig,ax = plt.subplots(figsize=(12,12),facecolor='w')
        shape_img1 = track_img_v[plane]
        shape_img2 = shower_img_v[plane]
        shape_img1=np.where(shape_img1>10.0,255.0,0.0).astype(np.uint8)
        shape_img2=np.where(shape_img2>10.0,100.0,0.0).astype(np.uint8)
        shape_img=shape_img1+shape_img2
        ax.imshow(shape_img,cmap='Greys',interpolation='none')
        nz_pixels=np.where(shape_img>0.0)
        print vtx.vtx2d_v.size()
        ax.plot(vtx.vtx2d_v[plane].x,vtx.vtx2d_v[plane].y,'*',color='cyan',markersize=20)
        
        ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        #ax.set_title("Vertex Type: %d"%vtx.type)
        print "<=============================================================================>"
        print "<===================End combined vertex ",vtxid," ==========================>"
        ax.grid()
        plt.show()



### Linear Track Cluster

In [ ]:
dm=mgr.DataManager()
colors=['red','green','blue','orange','magenta','cyan','pink']
colors*=10

dm=mgr.DataManager()
algoid=dm.ID("lineartrackfinder")
data=dm.Data(algoid,0)
lintrk_v=data.as_vector()

print "Found...",lintrk_v.size(),"...linear track clusters"

for strack_n,strack in enumerate(lintrk_v):
    print "<===================================================================================>"
    print "<===================Start LinearTrack number ",strack_n," ==========================>"

    # the only good one...
    
    e13d=strack.edge1
    e23d=strack.edge2
    print "Edges ",e13d,e23d
    for plane in xrange(3):
        strack2d = strack.get_cluster(plane)
        fig,ax=plt.subplots(figsize=(12,12),facecolor='w')
        shape_img1 = track_img_v[plane]
        shape_img2 = shower_img_v[plane]
        shape_img1=np.where(shape_img1>10.0,255.0,0.0).astype(np.uint8)
        shape_img2=np.where(shape_img2>10.0,100.0,0.0).astype(np.uint8)
        shape_img=shape_img1+shape_img2
        ax.imshow(shape_img,cmap='Greys',interpolation='none')
        plt.imshow(shape_img,cmap='Greys',interpolation='none')
        nz_pixels=np.where(shape_img>0.0)
        if strack2d.ctor.size()>0:
            ctor = [[pt.x,pt.y] for pt in strack2d.ctor]
            ctor.append(ctor[0])
            ctor=np.array(ctor)

            ax.plot(ctor[:,0],ctor[:,1],'-o',lw=3)

            e1=strack2d.edge1
            e2=strack2d.edge2

            ax.plot(e1.x,e1.y,'*',color='orange',markersize=20)
            ax.plot(e2.x,e2.y,'*',color='yellow',markersize=20)

            try:
                vtx2d=e13d.vtx2d_v[plane]
                pt=vtx2d.pt
                ax.plot(pt.x,pt.y,'o',color='green',markersize=40,alpha=0.5)
            except:
                pass

            try:
                vtx2d=e23d.vtx2d_v[plane]
                pt=vtx2d.pt
                ax.plot(pt.x,pt.y,'o',color='green',markersize=40,alpha=0.5)
            except:
                pass

        ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
        ax.set_xlabel('Time [6 ticks]',fontsize=20)
        ax.set_ylabel('Wire',fontsize=20)
        ax.set_aspect(0.8)
        ax.grid()
        plt.show()
        
    print "<===================End   LinearTrack number ",strack_n," ==========================>"
    print "<===================================================================================>"


In [ ]:
dm=mgr.DataManager()
algoid=dm.ID("showervertexestimate")
data=dm.Data(algoid,0)
print "Got ",data.as_vector().size()," shower 3D vertex estimate"
for vtxid,vtx3d in enumerate(data.as_vector()):
    print "<=============================================================================>"
    print "<===================Start concrete vertex ",vtxid," ==========================>"
    for plane in xrange(3):
        fig,ax=plt.subplots(figsize=(12,12),facecolor='w')
        shape_img1 = track_img_v[plane]
        shape_img2 = shower_img_v[plane]
        shape_img1=np.where(shape_img1>10.0,255.0,0.0).astype(np.uint8)
        shape_img2=np.where(shape_img2>10.0,100.0,0.0).astype(np.uint8)
        shape_img=shape_img1+shape_img2
        ax.imshow(shape_img,cmap='Greys',interpolation='none')
        nz_pixels = np.where(shape_img>0.0)
        
        cvtx=vtx3d.cvtx2d_v[plane]
        ax.plot(cvtx.center.x,cvtx.center.y,'o',color='red',markersize=10)
        circle_=matplotlib.patches.Circle((cvtx.center.x,cvtx.center.y),
                                          cvtx.radius,fc='none',ec='cyan',lw=5)
        for xs in cvtx.xs_v:
            ax.plot(xs.pt.x,xs.pt.y,'o',color='orange',markersize=10)

        ax.add_patch(circle_)
        ax.set_aspect(1.0)
        ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
        ax.set_xlabel('Time [6 ticks]',fontsize=20)
        ax.set_ylabel('Wire',fontsize=20)
        ax.set_title("Vertex Type = %d"%vtx3d.type,fontsize=30)
        ax.set_aspect(0.8)
        plt.show()
    print "<===================end concrete vertex ",vtxid," ============================>"
    print "<=============================================================================>"

### Shower + Track Particle Clusters

In [ ]:
#New VertexCluster
algoid=dm.ID("showervertexestimate")
data=dm.Data(algoid,0)
vtx_data=dm.Data(algoid,0).as_vector()

for vtxid,vtx in enumerate(vtx_data):
    vtxid+=1
    print "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~START VERTEX~~~~~~~~~~~~~~~~~~~~~~~~~"
    for plane in xrange(3):
        print "+++++++++++++++++++++Plane",plane
        trk_algoid=dm.ID("shower_vertexparticleclustermaker1")
        trk_par_data=dm.Data(trk_algoid,plane)

        trk_ass_t = np.array(assman.GetManyAss(vtx,trk_par_data.ID()))
        if trk_ass_t.size==0:
            print "No Associated track particles to vertex type: ",vtx.type
            print "============================================="
            continue

        shr_algoid=dm.ID("shower_vertexparticleclustermaker2")
        shr_par_data=dm.Data(shr_algoid,plane)

        shr_ass_t = np.array(assman.GetManyAss(vtx,shr_par_data.ID()))
        if shr_ass_t.size==0:
            print "No Associated shower particles to vertex type: ",vtx.type
            print "============================================="
            continue
        
        fig,ax = plt.subplots(figsize=(12,12),facecolor='w')
        shape_img1 = track_img_v[plane]
        shape_img2 = shower_img_v[plane]
        shape_img1=np.where(shape_img1>10.0,255.0,0.0).astype(np.uint8)
        shape_img2=np.where(shape_img2>10.0,100.0,0.0).astype(np.uint8)
        shape_img=shape_img1+shape_img2
        ax.imshow(shape_img,cmap='Greys',interpolation='none')
        nz_pixels=np.where(shape_img>0.0)

        trk_par_data_v=trk_par_data.as_vector()
        for id_ in trk_ass_t:
            ctor=[[pt.x,pt.y] for pt in trk_par_data_v[id_]._ctor]
            if len(ctor)==0: continue
            ctor.append(ctor[0])
            ctor=np.array(ctor)
            ax.plot(ctor[:,0],ctor[:,1],'-',lw=2,alpha=0.8)

        shr_par_data_v=shr_par_data.as_vector()
        for id_ in shr_ass_t:
            ctor=[[pt.x,pt.y] for pt in shr_par_data_v[id_]._ctor]
            if len(ctor)==0: continue
            ctor.append(ctor[0])
            ctor=np.array(ctor)
            ax.plot(ctor[:,0],ctor[:,1],'-',lw=2,alpha=0.8)

        ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
        ax.set_title("Vertex type=%d"%vtx.type,fontweight='bold',fontsize=20)
        plt.show()
        print "============================================="
    print "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~END VERTEX~~~~~~~~~~~~~~~~~~~~~~~~~"


### Combined Vertices

In [ ]:
#New VertexCluster
algoid=dm.ID("combinedvertexanalysis")
data=dm.Data(algoid,0)
vtx_data=dm.Data(algoid,0).as_vector()

for vtxid,vtx in enumerate(vtx_data):
    print "<=============================================================================>"
    print "<===================Start combined vertex ",vtxid," ==========================>"
    for plane in xrange(3):
        fig,ax = plt.subplots(figsize=(12,12),facecolor='w')
        shape_img1 = track_img_v[plane]
        shape_img2 = shower_img_v[plane]
        shape_img1=np.where(shape_img1>10.0,255.0,0.0).astype(np.uint8)
        shape_img2=np.where(shape_img2>10.0,100.0,0.0).astype(np.uint8)
        shape_img=shape_img1+shape_img2
        ax.imshow(shape_img,cmap='Greys',interpolation='none')
        nz_pixels=np.where(shape_img>0.0)

        par_data=dm.Data(algoid,plane+1)

        ass_t = np.array(assman.GetManyAss(vtx,par_data.ID()))
        if ass_t.size==0:continue

        par_data_v=par_data.as_vector()
        for id_ in ass_t:
            ctor=[[pt.x,pt.y] for pt in par_data_v[id_]._ctor]
            if len(ctor)==0: continue
            ctor.append(ctor[0])
            ctor=np.array(ctor)
            ax.plot(ctor[:,0],ctor[:,1],'-',lw=2,alpha=0.8)

        ax.plot(vtx.vtx2d_v[plane].pt.x,vtx.vtx2d_v[plane].pt.y,'*',color='cyan',markersize=20)
        ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
        ax.set_title("Vertex Type: %d"%vtx.type)
        plt.show()
    print "<=============================================================================>"
    print "<===================End combined vertex ",vtxid," ==========================>"

